### 시계열 검정을 위한 함수 노트북 

- 만든 함수를 편하게 쓰기 위해서 한 노트북에 다 저장해놓고 %run ~.ipynb를 통해 함수를 한번에 불러온다.
- 노트북 작성의 목적은 jupyter lab을 벗어나지 않은채 시계열 분석 및 신경망 학습을 할 수 있도록 데이터를 불러오거나 분석을 위한 함수들을 모아놓기이다.
- 실제 주가를 분석하고 예측이 주목적이므로 통계적 개념은 최대한 간략하게 적도록 한다,


### 함수 목차 
####    1. 상장된 회사명을 입력하면 코드를 반환해주는 함수
####    2. 시계열 안정성 검정 함수 


In [27]:
import pandas as pd 
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as pdd
from datetime import datetime
import FinanceDataReader as fdr
import datetime
from statsmodels.tsa.stattools import adfuller,pacf, acf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from arch.unitroot import PhillipsPerron  #pip install arch

### 1. 상장된 회사명을 입력하면 코드를 반환해주는 함수 

In [20]:
code_list = fdr.StockListing('KRX') # 한국거래소에 상장된 기업 정보 


def get_code(name) :
    code = code_list[code_list['Name'] == name].values[0][0]
    print(name,":", code)
    return code

### 2. 시계열 안정성 검정 함수 

- 시계열 안정성을 검정하기 위한 함수이다. 
- 시계열 안정성을 검정하기 위해서 ADF(확장된 Dickey Fuller) test와 PP(phillips Peron) test 를 실행한다.
- AR모형 확률과정에서 고유값의 길이가 1이상일 경우 과거의 영향이 발산하는 형태이므로 불안정한 시계열이 된다.
- 고유값의 길이가 1일때 단위근이라 하는데 단위근이 1인지 통계적 검정을 통해 시계열의 안정성을 검정한다.(이를 DF test라 한다.)
- DF test의 문제점은 시계열에 단위근이 존재할 경우 잔차항의 iid 가정이 파괴된다는 점이다.
- 이를 보완하기 위해 모형에 차분된 시계열의 과거항을 포함시키는 방식으로 검정을 하는 방법이 확장된 DF test이다. (ADF test)
- pp test란 오차항의 자기 상관이나 이분산이 존재할 떄 사용하는 비모수적 검정 방법이다. 

In [26]:
def test_stationarity(df, std_plot = True):  
    MA5 = df.rolling(window = 5).mean() #5일 이동평균선
    MA10 = df.rolling(window = 10).mean() # 10일 이동평균선
    std10= df.rolling(window = 10).std() # 10일 이동분산선

    plt.plot(df, color = 'blue', label = 'Original') #종가 그래프
    plt.plot(MA10, color = 'red', label = 'Rolling Mean(10)')
    if std_plot:
        plt.plot(std10, color = 'black', label = 'Rolling std(5)')
        
    plt.legend(loc = 'best')
    plt.title('Rolling Mean and Std Deviation')
    
    dftest = adfuller(df, autolag='AIC') # ADF test(단위근 검정)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print('ADF Test result')
    print (dfoutput)

    print('=======================================')
    pp = PhillipsPerron(df)
    print(pp.summary().as_text())
    
    plot_acf(df)
    plot_pacf(df)
    plt.show(block = False)
